Get feeddown yield of non-prompt D0s.

In [2]:
import numpy as np
from root_numpy import hist2array, array2hist
import ROOT as rt
import matplotlib.pyplot as plt

Welcome to JupyROOT 6.14/06


In [3]:
# Ingredients
# D0 efficiency
yield_gen_pr = [] # yield of generated prompt D0
yield_rec_pr = [] # yield of reconstructed prompt D0
yield_gen_fd = [] # yield of generated non-prompt D0
yield_rec_fd = [] # yield of reconstructed non-prompt D0
yield_sim_fd = [] # MC distribution of yields of non-prompt-D0 jets (ptD vs ptJetGen)
response_jets = [] # Response matrix for jets (ptJetRec vs ptJetGen)

# Binning
Nbins_ptJet = 100
range_ptJet = [0, 100]
Nbins_ptD = 40
range_ptD = [0, 40]

# Let's assume 4 bins in ptD and 3 bins in ptJetGen and 2 bins in ptJetRec
ratio_eff = np.array([1, 2, 3, 4])
yield_sim_fd = np.array([[1, 2, 3, 4], [1, 2, 1, 2], [4, 3, 2, 1]])
response_jets = np.array([[1, 1, 1], [2, 2, 2]])
yield_sim_fd_eff = yield_sim_fd.dot(ratio_eff)
yield_sim_fd_eff_smeared = response_jets.dot(yield_sim_fd_eff)
print("Efficiency ratio")
print(ratio_eff)
print("Simulated ptD vs ptJetGen")
print(yield_sim_fd)
print("Product")
print(yield_sim_fd_eff)
print("Response ptJetGen vs ptJetRec")
print(response_jets)
print("Product")
print(yield_sim_fd_eff_smeared)

Efficiency ratio
[1 2 3 4]
Simulated ptD vs ptJetGen
[[1 2 3 4]
 [1 2 1 2]
 [4 3 2 1]]
Product
[30 16 20]
Response ptJetGen vs ptJetRec
[[1 1 1]
 [2 2 2]]
Product
[ 66 132]


In [31]:
def get_bins(axis):
    return np.array([axis.GetBinLowEdge(i) for i in range(1, axis.GetNbins() + 2)])

def RebinHistogram2D(hisOld, xarray, yarray):
    hisNew = rt.TH2D(hisOld.GetName() + "_rebin", hisOld.GetTitle(), len(xarray) - 1, xarray, len(yarray) - 1, yarray)
    xaxis = hisOld.GetXaxis()
    yaxis = hisOld.GetYaxis()
    for j in range(1, yaxis.GetNbins() + 1):
        for i in range(1, xaxis.GetNbins() +1):
            hisNew.Fill(xaxis.GetBinCenter(i), yaxis.GetBinCenter(j), hisOld.GetBinContent(i, j))
    return hisNew


path_to_mc = "/home/vkucera/HFjets/MachineLearningHEP/Notebooks/histos.root"
path_to_eff = "/home/vkucera/HFjets/output/D0kINT7HighMultwithJets/vAN-20190810_ROOT6-1/pp_mc_prodD2H/resultsMBjetvspt/efficienciesD0ppMBjetvspt.root"
path_to_response = "/home/vkucera/HFjets/MachineLearningHEP/Notebooks/histos.root"
file_mc = rt.TFile.Open(path_to_mc, "read")
file_eff = rt.TFile.Open(path_to_eff, "read")
file_response = file_mc

# Get efficiency
eff_pr = file_eff.Get("eff_mult0")
print("X bins", get_bins(eff_pr.GetXaxis()))
arr_eff_pr = hist2array(eff_pr)
print(arr_eff_pr)
arr_eff_fd = arr_eff_pr
arr_eff_ratio = arr_eff_fd / arr_eff_pr
print(arr_eff_ratio)

# Get jet response
his_response = file_response.Get("hisRespJet")
binsx_response = get_bins(his_response.GetXaxis())
binsy_response = get_bins(his_response.GetYaxis())
print("X bins", binsx_response)
print("Y bins", binsy_response)
arr_response = hist2array(his_response).T
#print(arr_response)
#print(np.array_equal(binsx_response, binsy_response))

# Get ptD vs ptJet
his_sim_fd = file_mc.Get("hisPtDPtJet")
print("X bins", get_bins(his_sim_fd.GetXaxis()))
print("Y bins", get_bins(his_sim_fd.GetYaxis()))
his_sim_fd_rebin = RebinHistogram2D(his_sim_fd, get_bins(eff_pr.GetXaxis()), get_bins(his_response.GetYaxis()))
print("X bins", get_bins(his_sim_fd_rebin.GetXaxis()))
print("Y bins", get_bins(his_sim_fd_rebin.GetYaxis()))
arr_sim_fd = hist2array(his_sim_fd_rebin).T

# Multiply
arr_sim_fd_eff = arr_sim_fd.dot(arr_eff_ratio)
arr_sim_fd_eff_smeared = arr_response.dot(arr_sim_fd_eff)
print(arr_sim_fd_eff_smeared)

histest = rt.TH2D("histest", "histest", 4, 0, 4, 3, 0, 3)
arrtest = hist2array(histest).T
print(arrtest)

X bins [ 2.  3.  4.  5.  6.  8. 12. 24.]
[0.05315889 0.13513368 0.13775475 0.20771904 0.2554938  0.29534066
 0.3409679 ]
[1. 1. 1. 1. 1. 1. 1.]
X bins [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100.]
Y bins [  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64